<a href="https://colab.research.google.com/github/ougrid/my-knowledge-resource/blob/master/Hyperdimensional_Computing_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hyperdimensional Computing (HDC)

**Author:** Prachya Boonkwan (NECTEC, Thailand)

## URL --> `https://tinyurl.com/4ea28dt7`

## Libraries

We will use TorchHD for hyperdimensional computing. More information can be found in its [Github](https://github.com/hyperdimensional-computing/torchhd).

In [ ]:
!pip install --quiet torch
!pip install --quiet torch-hd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.3/345.3 kB 5.0 MB/s eta 0:00:00


In [ ]:
import torch as T
import torchhd as H

## Case Study

We will create a HV (hypervector) representation for the following table.

| Row ID | Fruits | Weights | Seasons |
|-------:|:------:|--------:|:-------:|
| 1      | Apple  | 149.0   | Autumn  |
| 2      | Lemon  | 70.5    | Winter  |
| 3      | Mango  | 173.2   | Summer  |

Next, we will get access to each row and each column of the table using hyperdimensional computing.

## Defining the Item Space

Let's first define the item space for representing this table.

- **Fruits:** There are three symbols -- `apple`, `lemon`, and `mango`.
- **Weight:** We quantize a weight into 10 levels with the minimum of 0 and the maximum of 200.
- **Seasons:** There are four symbols -- `spring`, `summer`, `autumn`, and `winter`.

According to their natural characteristics, each of these vector spaces will be initialized in different fashions.

Generally, there are three common methods of HV initialization.

- *Random initialization:* This method guarantees that all HVs are different from each other. Therefore, each HV is similar to nothing but itself.

- *Level initialization:* This method guarantees that the similarity between two neighboring HVs changes in a constant rate.

- *Circular initialization:* This method guarantees that the similarity between two neighboring HVs are periodic.

<img width="50%" src="https://torchhd.readthedocs.io/en/stable/_images/basis-hvs.png">

Here, the fruit space is initialized with the random method, the weight space with the level method, and the season space with the circular method, respectively.

In [ ]:
dimensions = 10_000

# Fruits: apple, lemon, mango
fruits = H.random(3, dimensions)
(apple, lemon, mango) = fruits
fruit_symbols = ['apple', 'lemon', 'mango']

# Weights: [0, 200] quantized to the scales of 10
weights = H.embeddings.Level(10, dimensions, low=0, high=200)

# Seasons: spring, summer, autumn, winter
seasons = H.circular(4, dimensions)
(spring, summer, autumn, winter) = seasons
season_symbols = ['spring', 'summer', 'autumn', 'winter']

Moreover, we define the symbols for columns and rows as follows.

- **Columns:** There are three columns.
- **Row IDs:** We will have at most 10 rows.

In [ ]:
# Columns: fruits, weights, seasons
columns = H.random(3, dimensions)
(col1, col2, col3) = columns

# Row IDs: maximum 10 IDs
rowids = H.random(10, dimensions)

## HV Representation

### Data Rows

The HV representation for each column $c_i$ and its value $v_i$ can be easily computed as follows.
$$
    \mathbf{c}_i \otimes \mathbf{v}_i
$$
where $\mathbf{c}_i$ and $\mathbf{v}_i$ are the HVs of the column $c_i$ and value $v_i$, respectively, and $\otimes$ is the binding operation (i.e. elementwise multiplication).

Therefore, the HV representation for the $i$-th row consisting of three columns (fruit $f_i$, weight $w_i$, and season $s_i$) can be computed below:
$$
\begin{eqnarray}
    \mathbf{r}_i & = & (\mathbf{c}_1 \otimes \mathbf{f}_i) \\
    &   & {} \oplus (\mathbf{c}_2 \otimes \mathbf{w}_i) \\
    &   & {} \oplus (\mathbf{c}_3 \otimes \mathbf{s}_i)
\end{eqnarray}
$$
where $\mathbf{f}_i$, $\mathbf{w}_i$, and $\mathbf{s}_i$ are HVs for the fruit $f_i$, weight $w_i$, and season $s_i$, respectively, and $\oplus$ is the bundling operation (i.e. addition).

In [ ]:
def create_row(fruit, weight, season):
    r_col1 = col1.bind(fruit)
    r_col2 = col2.bind(weights(T.FloatTensor([149.0])))
    r_col3 = col3.bind(season)
    row = r_col1.bundle(r_col2).bundle(r_col3)
    return row

### Table

Once we construct a HV representation for each row, we can compose a table out of them. First, each row has to be bound with its row ID $\mathbf{i}_i$.
$$
    \mathbf{i}_i \otimes \mathbf{r}_i
$$

In [ ]:
row1 = create_row(apple, 149.0, autumn).bind(rowids[0])
print(f'row1 = {row1}')

row1 = MAPTensor([[ 1.,  1.,  1.,  ...,  3., -1., -1.]])


In [ ]:
row2 = create_row(lemon, 70.5, winter).bind(rowids[1])
print(f'row2 = {row2}')

row2 = MAPTensor([[-1., -1.,  3.,  ...,  1.,  3.,  1.]])


In [ ]:
row3 = create_row(mango, 173.2, summer).bind(rowids[2])
print(f'row3 = {row3}')

row3 = MAPTensor([[-1., -1.,  1.,  ..., -1.,  1., -1.]])


Next, a table is in fact a bundle of bound rows.
$$
\begin{eqnarray}
    \mathbf{t} & = & \bigoplus_{i=1}^N \left[ \mathbf{i}_i \otimes \mathbf{r}_i \right]
\end{eqnarray}
$$

In [ ]:
table = row1.bundle(row2).bundle(row3)
print(f'table = {table}')

table = MAPTensor([[-1., -1.,  5.,  ...,  3.,  3., -1.]])


## Getting Access to the Table

### Accessing a Data Row

To get access to a data row, we simply bind the table with a HV of the specified row ID $\mathbf{i}_k$.
$$
\begin{eqnarray}
    \mathbf{i}_k \otimes \mathbf{t} & = & \mathbf{i}_k \otimes \left( \bigoplus_{i = 1}^{N} \left[ \mathbf{i}_i \otimes \mathbf{r}_i \right] \right) \\
    & = & \mathbf{i}_k \otimes ( \mathbf{i}_k \otimes \mathbf{r}_k) \oplus \textrm{noise} \\
    & = & \mathbf{r}_k \oplus \textrm{noise} \\
    & \approx & \mathbf{r}_k
\end{eqnarray}
$$

In [ ]:
def get_row(table, rowid):
    return table.bind(rowids[rowid])

In [ ]:
my_row = get_row(table, 1)
print(f'my_row = {my_row}')

my_row = MAPTensor([[-1., -1., -5.,  ...,  3., -3., -1.]])


### Accessing a Column of the Data Row

Once we obtain a data row, we can get access to each column by binding with the column HV $\mathbf{c}_k$.
$$
\begin{eqnarray}
    \mathbf{c}_k \otimes \mathbf{r} & = & \mathbf{c}_k \otimes \left( \bigoplus_{i = 1}^{N} \left[ \mathbf{c}_i \otimes \mathbf{v}_i \right] \right) \\
    & = & \mathbf{c}_k \otimes ( \mathbf{c}_k \otimes \mathbf{v}_k) \oplus \textrm{noise} \\
    & = & \mathbf{v}_k \oplus \textrm{noise} \\
    & \approx & \mathbf{v}_k
\end{eqnarray}
$$

In [ ]:
def get_column(row, colid):
    col_vec = columns[colid]
    val_vec = row.bind(col_vec)
    return val_vec

In [ ]:
val_vec = get_column(my_row, 0)
print(val_vec)

MAPTensor([[ 1., -1.,  5.,  ...,  3.,  3.,  1.]])


## Symbol Retrieval

Finally, once we obtain a HV representation, we can always retrieve its corresponding symbol from the item space.

Suppose we have a HV space $\{ \mathbf{h}_1, \mathbf{h}_2, \mathbf{h}_3, \ldots, \mathbf{h}_K \}$ for the symbol set $\{ s_1, s_2, s_3, \ldots, s_K \}$. We will compare the input HV $\mathbf{v}$ against each HV in the space with the cosine similarity. The HV $\mathbf{h}_{i^*}$ closest to $\mathbf{v}$ will be chosen, resulting in the $i^*$-th symbol getting retrieved.
$$
\begin{eqnarray}
    i^* & = & \arg \max_i \mathrm{cossim}(\mathbf{h}_i, \mathbf{v}) \\
    & = & \arg \max_i \frac{\mathbf{h}_i \cdot \mathbf{v}}{||\mathbf{h}_i|| \times ||\mathbf{v}||} \\
\end{eqnarray}
$$

In [ ]:
def find_itemid(item_matrix, val_vec):
    sim_matrix = item_matrix.cosine_similarity(val_vec)
    return sim_matrix.argmax()

In [ ]:
fruit_id = find_itemid(fruits, val_vec)
print(f'fruit = {fruit_symbols[fruit_id]}')

fruit = lemon
